<a href="https://colab.research.google.com/github/sanu0711/langChain-experiments/blob/main/Chain_LangChain.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# 📌 Summary: Choosing the Right Chain


In [ ]:
!pip install -qU langchain-openai

# LLMChain (Basic Single-Step Chain) (Stop support on wards need to use pipe insted)
🔹 When to Use:

* When you only need a simple prompt and response.

* Ideal for text generation, translations, and question answering.

In [ ]:
from langchain.chains import LLMChain
from langchain_openai import ChatOpenAI
from langchain.prompts import PromptTemplate
from google.colab import userdata

llm = ChatOpenAI(
    api_key = userdata.get('OPENAI_API_KEY'),
    model="gpt-4o",
    # temperature=0, # temperature controls the randomness of the output
    # max_tokens=None, # max_tokens controls the maximum number of tokens in the output
    # timeout=None, # timeout controls the maximum time to wait for the API to respond
    # max_retries=2,  # max_retries controls the maximum number of retries to attempt if the API fails
    #base_url="...", # base_url is the base URL for the OpenAI API
)
prompt = PromptTemplate.from_template("What is the best name for a company that sells {product}?")




chain = LLMChain(
    llm=llm, # llm is an LLM object
    prompt=prompt, # prompt is a PromptTemplate
    verbose=True, # verbose is a boolean that controls whether to print the prompt
    output_key="company_name", # output_key is a string that controls the output key
    # stop=["\n"], # stop is a list of strings that controls when to stop

    )
# response = chain.run("organic skincare")
response = chain.invoke({"product": "organic skincare"})
response



# chain = prompt | llm
# response = chain.invoke({"product": "organic skincare"})
# response



> Entering new LLMChain chain...
Prompt after formatting:
What is the best name for a company that sells organic skincare?

> Finished chain.


{'product': 'organic skincare',
 'company_name': "Choosing the best name for your organic skincare company involves capturing the essence of your brand, resonating with your target audience, and setting yourself apart from competitors. Here are a few suggestions:\n\n1. **PureEssence Naturals**\n2. **EcoGlow Skincare**\n3. **GreenGoddess Beauty**\n4. **NatureNurture Organics**\n5. **Earthly Elixirs**\n6. **FreshAura Botanicals**\n7. **WildBloom Skincare**\n8. **RadiantRoots Organics**\n9. **OrganicOasis Beauty**\n10. **VerdantVibe Naturals**\n\nWhen selecting a name, ensure it is available as a domain name and isn't trademarked by another company. It's also beneficial to test it with potential customers to gauge their reaction."}

# SimpleSequentialChain (Basic Step-by-Step Execution)
🔹 When to Use:

* When one chain’s output is needed as input for the next.

* Example: Generating a blog post **title → summary → keywords.**

* **Not Returns Intermediate Steps**

In [ ]:
from langchain.chains import SimpleSequentialChain, LLMChain
from langchain_openai import ChatOpenAI
from langchain.prompts import PromptTemplate


# First Chain: Generate a product name
first_chain = LLMChain(
    llm=llm,
    prompt=PromptTemplate.from_template("Suggest a product name for {category}."),
)

# Second Chain: Describe the generated product name
second_chain = LLMChain(
    llm=llm,
    prompt=PromptTemplate.from_template("Describe the product {product_name}."),
)

# Simple Sequential Chain
chain = SimpleSequentialChain(
    chains=[first_chain, second_chain]
)

# Invoke Chain
response = chain.invoke("organic skincare")
print(response)


{'input': 'organic skincare', 'output': '"PureGlow Naturals" is likely a product line focused on providing natural and organic skincare and beauty solutions. Such a brand usually emphasizes the use of high-quality, all-natural ingredients that aim to enhance skin health and appearance without the use of harsh chemicals, synthetic fragrances, or artificial additives. Products from PureGlow Naturals might include items such as facial cleansers, moisturizers, serums, and masks, formulated with plant-based ingredients, essential oils, and botanical extracts known for their nourishing and healing properties. This brand may appeal to consumers looking for clean beauty options that are environmentally sustainable and cruelty-free. However, without specific information or context, the details about this particular product line might vary.'}


# SequentialChain (Advanced Multi-Step Execution)

🔹 When to Use:

* When multiple inputs/outputs are required.

* Example: Generating summary, tweet, and email text from an article.

* **Returns Intermediate Steps**

In [ ]:
from langchain.chains import SequentialChain, LLMChain
from langchain_openai import ChatOpenAI
from langchain.prompts import PromptTemplate



# First Chain: Generate a product name
first_chain = LLMChain(
    llm=llm,
    prompt=PromptTemplate.from_template("Suggest a product name for {category}."),
    output_key="product_name",  # Store output as "product_name"
)

# Second Chain: Describe the generated product name
second_chain = LLMChain(
    llm=llm,
    prompt=PromptTemplate.from_template("Describe the product {product_name}."),
    output_key="description",  # Store output as "description"
)

# Creating a SequentialChain
chain = SequentialChain(
    chains=[first_chain, second_chain],  # Run chains in order
    input_variables=["category"],  # Initial input
    output_variables=["product_name", "description"],  # Final outputs
    return_all=True  # ✅ This returns intermediate steps
)

# Invoke Chain
response = chain.invoke({"category": "organic skincare"})
response


{'category': 'organic skincare',
 'product_name': '"PureEarth Essentials"',
 'description': 'As of my last update, there isn\'t a specific product known as "PureEarth Essentials" that I can provide details about. However, based on the name, it might refer to a brand or product line focused on eco-friendly or natural goods. Typically, these kinds of products could include items like organic skincare, household products, or sustainable lifestyle goods. If "PureEarth Essentials" has been released or gained recognition after my last update, I recommend checking the company\'s website or recent product listings for accurate and up-to-date information.'}

# Router Chains (Dynamic Routing for Complex Workflows)

Router Chains allow dynamic selection of chains or LLM prompts based on user input. Instead of a single static chain, Router Chains intelligently route inputs to different specialized chains.

✅ Better Handling of Diverse Inputs → Different questions get routed to different models/prompts.

✅ Efficient Resource Usage → Uses the right model for the right task.

✅ Scalability → Easily extendable to multiple chains.

## LLM Router Chain (MultiRouteChain)

* Uses an LLM to decide which chain to route to.

* Example: Directs math-related queries to a calculator and text-based queries to GPT.

## MultiPromptChain

* Routes user input to different prompt templates.

* Example: Routes translation queries to a translator prompt and summarization queries to a summarization prompt.

# Custom Python Functions (✅ Ultimate Flexibility) -- > RunnableLambda

* Best for: Mixing sequential and parallel chains, adding if-else conditions, or looping.

In [ ]:
from langchain.schema.runnable import RunnableLambda
from langchain_openai import ChatOpenAI
from langchain.prompts import PromptTemplate
from langchain.chains import LLMChain


# Define a custom function


first_chain = LLMChain(
    llm=llm,
    prompt=PromptTemplate.from_template("Suggest a product name for {category}."),
    output_key="product_name",
)

second_chain = LLMChain(
    llm=llm,
    prompt=PromptTemplate.from_template("Describe the product {product_name}."),
    output_key="description",
)

# Custom logic to format output
def combine_outputs(inputs):
    return {
        "summary": f"{inputs['product_name']} is a great product. {inputs['description']}",
        "detailed_output": inputs
    }

combine_chain = RunnableLambda(combine_outputs) # make function runnable in langchain

# Final runnable sequence
chain = first_chain | second_chain | combine_chain

chain.invoke({"category": "organic skincare"})



{'summary': '"Ethereal Glow Organics" is a great product. "Ethereal Glow Organics" sounds like a brand or product line that focuses on organic skincare or beauty products. While I don\'t have specific information about this product, based on the name, it likely emphasizes natural and organic ingredients aimed at enhancing skin\'s natural radiance and health. Such products often include elements like plant-based extracts, essential oils, or other organic components designed to nourish, hydrate, and rejuvenate the skin. If it\'s part of a larger lineup, the brand might pride itself on sustainable practices, eco-friendly packaging, and a commitment to clean beauty standards. If you’re interested in specifics, I recommend checking the brand\'s official website or product descriptions for more detailed information.',
 'detailed_output': {'category': 'organic skincare',
  'product_name': '"Ethereal Glow Organics"',
  'description': '"Ethereal Glow Organics" sounds like a brand or product lin

# RunnableParallel (Parallel Execution for Faster Performance)

* ✅ When multiple tasks do not depend on each other’s output.

* ✅ To speed up execution by running independent tasks at the same time.

* ✅ To combine multiple outputs in a single response.

In [ ]:
from langchain.schema.runnable import RunnableParallel
from langchain_openai import ChatOpenAI
from langchain.prompts import PromptTemplate
from langchain.chains import LLMChain


# Define independent chains
product_name_chain = LLMChain(
    llm=llm,
    prompt=PromptTemplate.from_template("Suggest a brand name for {category}."),
    output_key="product_name",
)

tagline_chain = LLMChain(
    llm=llm,
    prompt=PromptTemplate.from_template("Write a catchy tagline for a {category} brand."),
    output_key="tagline",
)

social_media_chain = LLMChain(
    llm=llm,
    prompt=PromptTemplate.from_template("Write a Twitter post for {category} brand."),
    output_key="social_post",
)

# Run all chains in parallel
parallel_chain = RunnableParallel(
    product_name=product_name_chain,
    tagline=tagline_chain,
    social_post=social_media_chain,
)

parallel_chain.invoke({"category": "organic skincare"})



{'product_name': {'category': 'organic skincare',
  'product_name': "Creating a brand name for an organic skincare line involves conveying purity, nature, and wellness. Here are a few suggestions:\n\n1. PurePetal\n2. EarthEssence\n3. GaiaGlow\n4. NatureNurture\n5. GreenGlimmer\n6. EcoElixir\n7. VitalVerde\n8. BloomBare\n9. OrganicOra\n10. FloraFusion\n\nIt's important to ensure that the name aligns with your brand values and is unique in the skincare market."},
 'tagline': {'category': 'organic skincare',
  'tagline': '"Nature\'s Glow, Purely You."'},
 'social_post': {'category': 'organic skincare',
  'social_post': '🌿✨ Embrace the beauty of nature with our all-natural, organic skincare line! Treat your skin to the purest ingredients, responsibly sourced and cruelty-free. Glow from the inside out with the power of nature! 💚 #OrganicBeauty #NaturalGlow #CrueltyFree #EcoFriendlySkincare 🌱💧'}}

# RunnableMap (Batch Processing on Multiple Inputs)

RunnableMap is used to process multiple inputs independently and return a dictionary of results. It is similar to RunnableParallel, but instead of executing everything together, each function gets its own specific input.

🔹 When to Use RunnableMap?

✅ When different tasks need different inputs.

✅ When you need to map multiple inputs to multiple outputs.

✅ When you want to structure results in a dictionary format.

In [ ]:
from langchain.schema.runnable import RunnableMap
from langchain_openai import ChatOpenAI
from langchain.prompts import PromptTemplate
from langchain.chains import LLMChain


# Define individual chains with different inputs
product_name_chain = LLMChain(
    llm=llm,
    prompt=PromptTemplate.from_template("Suggest a brand name for {category}."),
    output_key="product_name",
)

description_chain = LLMChain(
    llm=llm,
    prompt=PromptTemplate.from_template("Describe the product {product_name}."),
    output_key="description",
)

tagline_chain = LLMChain(
    llm=llm,
    prompt=PromptTemplate.from_template("Write a catchy tagline for a {category} brand."),
    output_key="tagline",
)

# RunnableMap to execute all chains independently with specific inputs
map_chain = RunnableMap({
    "product_name": product_name_chain,   # Takes category as input
    "description": description_chain,     # Takes product_name as input
    "tagline": tagline_chain              # Takes category as input
})

map_chain.invoke({
    "category": "organic skincare",
    "product_name": "GlowNaturals"
})




{'product_name': {'category': 'organic skincare',
  'product_name': 'Certainly! Here are a few suggestions for an organic skincare brand name:\n\n1. **PureSerene**\n2. **EcoGlow**\n3. **NaturaLuxe**\n4. **VerdantVisage**\n5. **BotanikEssence**\n6. **GreenAura**\n7. **OrganicaRadiance**\n8. **EarthElixir**\n9. **HerbaHarmony**\n10. **GaiaGlow**\n\nMake sure to check for trademark availability and domain accessibility if you decide to use one of these names.'},
 'description': {'category': 'organic skincare',
  'product_name': 'GlowNaturals',
  'description': "I'm sorry, but as of my last update in October 2023, I don't have specific information about a product called GlowNaturals. It's possible that it could be a niche or recent product not widely documented in my sources. If you can provide more detailed information about the product, such as its category (skincare, supplements, etc.) or its main features, I might be able to help more effectively. Alternatively, you could check the man

# 🚀 ConversationalRetrievalChain: Chatbots with Memory + Retrieval

ConversationalRetrievalChain is a specialized chain in LangChain that enables chatbots to remember previous interactions while retrieving relevant documents from a vector database (like FAISS, Chroma, Pinecone, etc.).

 **Why Use ConversationalRetrievalChain?**

✅ Memory: Keeps track of past conversations, allowing the chatbot to maintain context.

✅ Retrieval: Fetches relevant information from stored documents using a retriever (like vector databases).

✅ Conversational AI: Ideal for chatbots, customer support, and knowledge-based Q&A systems.

# Parsers in LangChain

## 1. PydanticOutputParser (Most Common for Structured Data)

Best for: Structured output with validation.

Why? Ensures correct data types and validation using Pydantic models.

Common Use Cases: Financial models, AI assistants, database interactions.

In [ ]:
from langchain.output_parsers import PydanticOutputParser
from pydantic import BaseModel, Field

# Define a structured output model
class InvestmentPlan(BaseModel):
    risk_level: str = Field(..., description="Risk tolerance level")
    suggested_assets: list[str] = Field(..., description="Recommended investment assets")

# Create the parser
parser = PydanticOutputParser(pydantic_object=InvestmentPlan)

# Format instructions to guide the LLM

formatted_instructions = parser.get_format_instructions()


print(formatted_instructions)


The output should be formatted as a JSON instance that conforms to the JSON schema below.

As an example, for the schema {"properties": {"foo": {"title": "Foo", "description": "a list of strings", "type": "array", "items": {"type": "string"}}}, "required": ["foo"]}
the object {"foo": ["bar", "baz"]} is a well-formatted instance of the schema. The object {"properties": {"foo": ["bar", "baz"]}} is not well-formatted.

Here is the output schema:
```
{"properties": {"risk_level": {"description": "Risk tolerance level", "title": "Risk Level", "type": "string"}, "suggested_assets": {"description": "Recommended investment assets", "items": {"type": "string"}, "title": "Suggested Assets", "type": "array"}}, "required": ["risk_level", "suggested_assets"]}
```


## 2. StructuredOutputParser (Common for Enforcing Formats)

Best for: Ensuring a consistent output structure.

Why? Useful when working with multiple LLM responses that must have the same format.

In [ ]:
from langchain.output_parsers import StructuredOutputParser
from langchain.prompts import PromptTemplate

# Define a format template
format_template = StructuredOutputParser.from_response_schemas([
    {"name": "name", "description": "The name of the user"},
    {"name": "age", "description": "The age of the user"}
])

# Get formatting instructions
format_instructions = format_template.get_format_instructions()
print(format_instructions)


The output should be a markdown code snippet formatted in the following schema, including the leading and trailing "```json" and "```":

```json
{
	"name": string  // The name of the user
	"age": string  // The age of the user
}
```
